# Retrieval-Augmented Generation (RAG) project from Scratch
- RAG combaine information retrieval with text generation to enhance language models (LMs) performance by incorporating external knhowledge sources
- By doing so, RAG enhance aplications in systems such as question answering, dialogue systems, and much more generated content.

## What is RAG?

- While chatbots can respond most common questions based on previous training with large datasets, it may lack on domain-specific knowlidge. In a real world example, we could ask to a chatbot "Which medicines am I allergic to?". An LLM withou a specific dataset can not answer such questions. To solve such problem, we need to change the application architecture by adding the domain knowlage that is not avaiable it to the chatbot.  RAG systems can be resumed as two principal components:

  - A retrieval model that search information on a external knowlegde source. This external knowladge source can be a database and many other machanism that store info.

  - A LM that generates responses using the retrieved knowledge





    

#  SIMPLE RAG

- Lets then create a Simple RAG system to ilustrate how composition of LLM models with domain knowladge can be of great impact.

- The structure will be:

1. Embeddind model: A pre-treined LM that converts input text into embeddings - vector representations that capture semantic meaning.

2. Vector database: A storage system for info and its corresponding embedding vectors. We will follow the tutorial and implement a in-memory databse builded from scratch

3. Chatbot: A model that generate answers using retrieved knowledge. We will choose some foudation model that can run easily with low resources

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving asian_farmer.txt to asian_farmer.txt


- To train the model, I will utilize a book of agriculture that was avaiable for free at the web. The info about the book are:

Title: Farmers of Forty Centuries
     or, Permanent Agriculture in China, Korea and Japan

Author: F. H. King


In [ ]:
pip install requests

In [ ]:
#import whisper

import requests

from transformers import AutoModelForCausalLM, AutoTokenizer

import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
#Load LLM model and tokenizer lazily
model_id = "Qwen/Qwen2.5-1.5B-Instruct"
model = None
tokenizer = None

def load_model():
    global model, tokenizer
    if model is None or tokenizer is None:
        logger.info("Loading model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")  # Will use GPU if available
        logger.info("Model and tokenizer loaded.")
    return model, tokenizer


In [ ]:
import requests


#Raw URL of the dataset with lots of info about cats
url = "https://huggingface.co/ngxson/demo_simple_rag_py/resolve/main/cat-facts.txt"

#Downloading the text file
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request failed

#Split it into lines
dataset = response.text.splitlines()

print(f'Loaded {len(dataset)} entries')


Loaded 150 entries


In [ ]:
with open('asian_farmer.txt', 'r') as file:
  dataset = file.read().splitlines()
print(f'Loaded {len(dataset)} entries')

Loaded 9847 entries


In [ ]:
from sentence_transformers import SentenceTransformer


In [ ]:

#Embedding model from Hugging Face Hub
EMBEDDING_MODEL_ID = 'BAAI/bge-small-en-v1.5'

embedding_model = SentenceTransformer(EMBEDDING_MODEL_ID)

VECTOR_DB = []

def add_chunk_to_database(chunk):
    embedding = embedding_model.encode(chunk).tolist()  # Convert to list of floats
    VECTOR_DB.append((chunk, embedding))


for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to database')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
Added chunk 4848/9847 to database
Added chunk 4849/9847 to database
Added chunk 4850/9847 to database
Added chunk 4851/9847 to database
Added chunk 4852/9847 to database
Added chunk 4853/9847 to database
Added chunk 4854/9847 to database
Added chunk 4855/9847 to database
Added chunk 4856/9847 to database
Added chunk 4857/9847 to database
Added chunk 4858/9847 to database
Added chunk 4859/9847 to database
Added chunk 4860/9847 to database
Added chunk 4861/9847 to database
Added chunk 4862/9847 to database
Added chunk 4863/9847 to database
Added chunk 4864/9847 to database
Added chunk 4865/9847 to database
Added chunk 4866/9847 to database
Added chunk 4867/9847 to database
Added chunk 4868/9847 to database
Added chunk 4869/9847 to database
Added chunk 4870/9847 to database
Added chunk 4871/9847 to database
Added chunk 4872/9847 to database
Added chunk 4873/9847 to database
Added chunk 4874/9847 to database
Added chunk 4875/9847 to databa

## Retrieval Function

- Now we will implement a retrieval function that will take a query and returns the top N most important chunks. This N chunks will be choosen using a cosine similarity.
 - The higher the cosine similarity between two vectors, the closer they are in the vector space and alike in mining they are

In [ ]:
#lets def the cosine similarity mechanism

def cosine_similarity(a, b):
  dot_product = sum([x*y for x, y in zip(a,b)])
  magnitude_a = sum([x ** 2 for x in a]) ** 0.5
  magnitude_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (magnitude_a * magnitude_b)

In [ ]:
# Finally, def the retrieve function

def retrieve(query, top_n=8):
  query_embedding = embedding_model.encode(query).tolist()
  #List to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # We sort in such a way that the chunks with higher similarity show up at the top of the list.
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]



# Test the RAG

In [ ]:
class AsianFarmChatbot:
    def __init__(self, retrieve_func, model_loader_func):
        """
        :param retrieve_func: function to retrieve knowledge, e.g., retrieve(query)
        :param model_loader_func: function to load model and tokenizer, e.g., load_model()
        """
        self.retrieve = retrieve_func
        self.load_model = model_loader_func
        self.model, self.tokenizer = self.load_model()

    def build_prompt(self, context_chunks):
        context_text = '\n'.join([f' - {chunk}' for chunk, _ in context_chunks])
        instruction = (
            "You are a helpful chatbot whose only mission is to provide closed information about asian farming facts.\n"
            "Use only the following pieces of context to answer the question.\n"
            "Don't make up any new information or ask more questions — just formulate concise answers with maximum length of 400 tokens:\n"
            f"{context_text}"
        )
        return instruction

    def generate_answer(self, prompt, max_length=400):
        inputs = self.tokenizer(prompt, return_tensors='pt').to(self.model.device)
        output_ids = self.model.generate(
            **inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.2,
            top_p=0.5,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.pad_token_id
        )
        answer = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return answer[len(prompt):].strip()

    def ask(self, query):
        retrieved_knowledge = self.retrieve(query)
        prompt = self.build_prompt(retrieved_knowledge)
        answer = self.generate_answer(prompt)
        return answer


In [ ]:
chatbot = AsianFarmChatbot(retrieve, load_model)
response = chatbot.ask("What are the main products of Asian farm?")
print(response)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In addition, the Chinese
 - agriculture has been known to use chemical fertilizers since the late
 - 19th century.

The permanent agriculture in these countries involves the cultivation of various crops such as water rice, dry land rice, and other agricultural crops across different regions within China and Japan. These crops serve multiple purposes including food production, clothing (apparel), and sometimes even livestock feed. Potassium, an essential element, is applied by both Japanese and Chinese farmers through natural methods like composting and chemical fertilizers introduced later in the 19th century. This practice ensures that the soil remains fertile and supports sustainable agricultural practices over time. 

---

In summary, the focus on permanent agriculture in China, Korea, and Japan revolves around cultivating diverse crops tailored to local climates and needs. While traditional methods have been used for centuries, modern advancements include the application of potassi